In [1]:
!mkdir /root/.config/kaggle
!mv kaggle.json /root/.config/kaggle/

In [4]:
import kagglehub

import kagglehub
featsqueeze = kagglehub.model_download('shusrith/cifar10_cnn_gradmask/PyTorch/default/1')
normal = kagglehub.model_download('shusrith/cifar10-cnn-residual/PyTorch/default/1')
gradmask = kagglehub.model_download('shusrith/cifar-10-cnn-gradmask/PyTorch/with-dropout/1')


  0%|          | 0.00/18.5M [00:00<?, ?B/s]
 38%|███▊      | 7.00M/18.5M [00:00<00:00, 72.3MB/s]
100%|██████████| 18.5M/18.5M [00:00<00:00, 74.4MB/s]



  0%|          | 0.00/18.5M [00:00<?, ?B/s]
 27%|██▋       | 5.00M/18.5M [00:00<00:00, 41.6MB/s]
100%|██████████| 18.5M/18.5M [00:00<00:00, 71.7MB/s]



  0%|          | 0.00/18.5M [00:00<?, ?B/s]
 38%|███▊      | 7.00M/18.5M [00:00<00:00, 52.8MB/s]
100%|██████████| 18.5M/18.5M [00:00<00:00, 45.0MB/s]


In [5]:
import torch.nn as nn
import torch


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return torch.relu(out)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = ResidualBlock(64, 128, stride=2)
        self.layer2 = ResidualBlock(128, 256, stride=2)
        self.layer3 = ResidualBlock(256, 512, stride=2)
        self.dropout = nn.Dropout(0.3)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc(x)

In [47]:
import torch

def stochastic_relu(x):
    noise = torch.randn_like(x) * 0.5
    return torch.relu(x + noise)


import torch.nn as nn
import torch


class ResidualBlock1(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock1, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False,
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels, out_channels, kernel_size=1, stride=stride, bias=False
                ),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        out = stochastic_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return stochastic_relu(out)


class CNN1(nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = ResidualBlock1(64, 128, stride=2)
        self.layer2 = ResidualBlock1(128, 256, stride=2)
        self.layer3 = ResidualBlock1(256, 512, stride=2)
        self.dropout = nn.Dropout(0.3)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 10)

    def forward(self, x):
        x = stochastic_relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        return self.fc(x)

In [39]:
import torchvision
train = torchvision.datasets.CIFAR10(root='./data',train=True, download=True)
transform = torchvision.transforms.ToTensor()
a = transform(train[0][0])
print(train[0][1])

Files already downloaded and verified
6


In [40]:
target_class = torch.tensor([9], dtype=torch.long, device="cuda")

In [41]:
model = CNN()
model.load_state_dict(torch.load(f"{normal}/cifar10_cnn.pth", weights_only=True))
model = model.to("cuda")

In [54]:
def fgsm_attack(model, input_image, target_class, epsilon=0.1):
    input_image = input_image.unsqueeze(0).to("cuda")
    input_image.requires_grad = True

    model.eval()
    output = model(input_image)

    loss = -torch.nn.functional.cross_entropy(output, target_class)
    model.zero_grad()
    loss.backward()

    if input_image.grad is None:
        raise ValueError("Gradients not computed. Ensure input_image.requires_grad = True before forward pass.")

    with torch.no_grad():
        perturbed_image = input_image + epsilon * input_image.grad.sign()
        perturbed_image = torch.clamp(perturbed_image, 0, 1)
        output1 = model(perturbed_image)
        pred = torch.nn.functional.softmax(output1, dim=1)
        pred = torch.argmax(pred)

    return loss.item(), perturbed_image, pred

In [55]:
fgsm_attack(model, a, target_class)

(-13.551685333251953,
 tensor([[[[0.3314, 0.2686, 0.2961,  ..., 0.7196, 0.6961, 0.4804],
           [0.0000, 0.0000, 0.1706,  ..., 0.3824, 0.5667, 0.3784],
           [0.1980, 0.0000, 0.2922,  ..., 0.3627, 0.3706, 0.3275],
           ...,
           [0.9157, 0.6882, 0.8765,  ..., 0.7275, 0.3196, 0.1078],
           [0.6059, 0.5784, 0.6294,  ..., 0.6216, 0.2804, 0.2255],
           [0.5941, 0.7588, 0.8020,  ..., 0.7471, 0.4922, 0.3824]],
 
          [[0.3431, 0.2804, 0.0882,  ..., 0.6176, 0.3902, 0.3863],
           [0.0000, 0.0000, 0.0000,  ..., 0.4451, 0.4255, 0.2412],
           [0.1941, 0.0000, 0.0059,  ..., 0.4294, 0.4294, 0.3863],
           ...,
           [0.7667, 0.5000, 0.7314,  ..., 0.4216, 0.0216, 0.2333],
           [0.6451, 0.5824, 0.4647,  ..., 0.6804, 0.3431, 0.3078],
           [0.6647, 0.6059, 0.6569,  ..., 0.8216, 0.5627, 0.4608]],
 
          [[0.3471, 0.2765, 0.2686,  ..., 0.3235, 0.3000, 0.3039],
           [0.1784, 0.1000, 0.0000,  ..., 0.1157, 0.2961, 0.1235],
  

In [56]:
model1 = CNN1()
model1.load_state_dict(torch.load(f"{gradmask}/cifar10_cnn_grad_masking.pth", weights_only=True))
model1 = model1.to("cuda")

In [57]:
fgsm_attack(model1, a, target_class)

(-11.181758880615234,
 tensor([[[[0.3314, 0.2686, 0.2961,  ..., 0.5196, 0.6961, 0.6804],
           [0.0000, 0.0000, 0.1706,  ..., 0.5824, 0.3667, 0.3784],
           [0.1980, 0.0000, 0.2922,  ..., 0.5627, 0.5706, 0.3275],
           ...,
           [0.7157, 0.8882, 0.8765,  ..., 0.7275, 0.1196, 0.3078],
           [0.8059, 0.5784, 0.6294,  ..., 0.6216, 0.2804, 0.2255],
           [0.7941, 0.7588, 0.8020,  ..., 0.9471, 0.6922, 0.3824]],
 
          [[0.1431, 0.0804, 0.2882,  ..., 0.4176, 0.3902, 0.5863],
           [0.1784, 0.0000, 0.1314,  ..., 0.2451, 0.2255, 0.4412],
           [0.1941, 0.0000, 0.2059,  ..., 0.4294, 0.2294, 0.3863],
           ...,
           [0.5667, 0.5000, 0.7314,  ..., 0.6216, 0.0216, 0.2333],
           [0.4451, 0.3824, 0.4647,  ..., 0.4804, 0.3431, 0.3078],
           [0.4647, 0.6059, 0.4569,  ..., 0.8216, 0.5627, 0.4608]],
 
          [[0.3471, 0.2765, 0.0686,  ..., 0.3235, 0.3000, 0.3039],
           [0.1784, 0.0000, 0.1000,  ..., 0.3157, 0.0961, 0.3235],
  

In [59]:
import numpy as np
import cv2

def to4bit(image):
    return torch.round(image * 15) / 15

def median_filter(image):
    image_np = (image.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    filtered_np = np.stack([cv2.medianBlur(image_np[:, :, c], 3) for c in range(3)], axis=-1)
    filtered_tensor = torch.tensor(filtered_np, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return filtered_tensor



import torchvision.transforms as transforms
import numpy as np
import cv2

transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        transforms.Lambda(to4bit),
        transforms.Lambda(median_filter)
    ]
)

transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
        transforms.Lambda(to4bit),
        transforms.Lambda(median_filter),
    ]
)

train = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform_train)
a, label = train[0]
label

Files already downloaded and verified


6

In [61]:
model = CNN()
model.load_state_dict(torch.load(f"{featsqueeze}/cifar10_cnn_grad_masking.pth", weights_only=True))
model = model.to("cuda")

In [63]:
fgsm_attack(model, a, target_class)

(-10.039913177490234,
 tensor([[[[0.1078, 0.0000, 0.0000,  ..., 0.1078, 0.1078, 0.0000],
           [0.1078, 0.0000, 0.0000,  ..., 0.1078, 0.1078, 0.0000],
           [0.1078, 0.0000, 0.0000,  ..., 0.1078, 0.1078, 0.0000],
           ...,
           [0.0333, 0.1000, 0.3667,  ..., 0.1078, 0.1078, 0.0000],
           [0.3667, 0.1667, 0.3667,  ..., 0.1078, 0.0000, 0.0000],
           [0.1667, 0.3667, 0.1667,  ..., 0.0000, 0.0000, 0.1078]],
 
          [[0.0000, 0.1078, 0.0000,  ..., 0.1078, 0.1078, 0.0000],
           [0.0000, 0.1078, 0.0000,  ..., 0.1078, 0.1078, 0.0000],
           [0.1078, 0.0000, 0.1078,  ..., 0.1078, 0.1078, 0.0000],
           ...,
           [0.3706, 0.5706, 0.5039,  ..., 0.1078, 0.1078, 0.1078],
           [0.5706, 0.6373, 0.5039,  ..., 0.1078, 0.1078, 0.1078],
           [0.5706, 0.5039, 0.5039,  ..., 0.0000, 0.0000, 0.1078]],
 
          [[0.1745, 0.1745, 0.1745,  ..., 0.3745, 0.3745, 0.1745],
           [0.3745, 0.3745, 0.1745,  ..., 0.3745, 0.3745, 0.1745],
  